# School

Bir **Warm-Up** etkinliği ve hızlı bir `SELECT` ifadesiyle başlayalım.


## Data
Bu URL'de bulunan `school.sqlite` database ile çalışacağız:
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/school.sqlite`

Dosyayı indirmek için aşağıdaki hücreyi çalıştırın:


In [29]:
!mkdir -p data
!curl -sSL https://wagon-public-datasets.s3.amazonaws.com/sql_databases/school.sqlite > data/school.sqlite
!ls -lh data

total 24
-rw-r--r--@ 1 gizemtotkanli  staff    12K 23 Ara 15:15 school.sqlite


## Database Discovery

Database'i keşfetmek için (database'i indirdikten sonra) VS Code SQLite extension'ını kullanabilirsiniz:

- `Ctrl-Shift-P` veya `Cmd-Shift-P` tuşlarına basın
- `SQLite: Open Database` görünene kadar yazmaya başlayın
- `Enter`
- Açılan listeden database dosyasını seçin
- Sol alt köşede `SQLITE EXPLORER` butonuna tıklayın

Aşağıdaki soruları cevaplayın:

- Kaç tane table var?
- Her table için column'lar neler?

Schema ile rahat ettiğinizde bir SQL Query yazalım.



## Setup

Pandas ve sqlite3 ihtiyacımız olan her şey :-)

In [30]:
import pandas as pd
from sqlite3 import connect

In [31]:
with connect("data/school.sqlite") as conn:
    tables = pd.read_sql("""
        SELECT name
        FROM sqlite_master
        WHERE type='table'
        ORDER BY name;
    """, conn)

tables

,name
0,sqlite_sequence
1,students


In [32]:
with connect("data/school.sqlite") as conn:
    # students tablosunun kolonları
    pd.read_sql("PRAGMA table_info(students);", conn)[["name", "type"]]

In [33]:
from IPython.display import display

with connect("data/school.sqlite") as conn:
    cols = pd.read_sql("PRAGMA table_info(students);", conn)[["name", "type"]]

display(cols)

,name,type
0,id,INTEGER
1,first_name,TEXT
2,last_name,TEXT
3,birth_city,TEXT


## Paris Students

👉 `Paris` şehrinden tüm students kayıtlarını seçmek için bir SQL Query yaz.

In [34]:
query_students = """
SELECT *
FROM students
WHERE birth_city = 'Paris'
"""

In [35]:
with connect("data/school.sqlite") as conn:
    df = pd.read_sql(query_students, con=conn)

df.head()

,id,first_name,last_name,birth_city
0,1,Oran,Southern,Paris
1,6,Bertha,Brook,Paris
2,7,Neha,Salazar,Paris
3,8,Ignacy,Casey,Paris
4,10,Shirley,Mayer,Paris


## Dynamic queries

Harika, Paris öğrencilerini aldık.

Ama bu sorgunun **dinamik** olması (yani herhangi bir şehirle çalışması) güzel olmaz mıydı?

👉 Sorguyu, herhangi bir şehir için kullanabileceğimiz şekilde yeniden yazın. Aşağıdaki kodda, DataFrame’i yüklerken bunu nasıl kullanacağımızı görebilirsiniz.


In [36]:
query_students = """
SELECT *
FROM students
WHERE birth_city = ?
"""

In [37]:
city = "London"

with connect("data/school.sqlite") as conn:
    df = pd.read_sql(query_students, con=conn, params=[city])

df.head()

,id,first_name,last_name,birth_city
0,2,Safa,Lugo,London
1,5,Rick,Broadhurst,London


Sonuç beklediğiniz gibi göründüğünde, sorgunuzu test etmek için aşağıdaki hücreyi çalıştırın.


In [38]:
from nbresult import ChallengeResult

result = ChallengeResult(
    "school",
    query=query_students
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/gizemtotkanli/.pyenv/versions/workintech_current/bin/python
cachedir: .pytest_cache
rootdir: /Users/gizemtotkanli/data_back_to_school_query/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 5 items

test_school.py::TestSchool::test_barcelona PASSED                        [ 20%]
test_school.py::TestSchool::test_berlin PASSED                           [ 40%]
test_school.py::TestSchool::test_brussels PASSED                         [ 60%]
test_school.py::TestSchool::test_london PASSED                           [ 80%]
test_school.py::TestSchool::test_paris PASSED                            [100%]

============================== 5 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/school.pickle

git commit -m 'Completed school step'

git push origin master



## Key learning points

- Bir database üzerinde query yazmak için Pandas kullanın
- Dinamik queries yazmak için parameter substitution kullanın